# WDBC Data Sets Analysis

The following jupyter notebook provides summary of the analysis undertaken on the `WDBC` data set.

## Download

Download available files.
 1. List available files from the website
 2. Create absolute url paths and download files to temp folder
*The snippet useful if re-running the netbook and saves hassle with sourcing raw data*

In [4]:
# Modules
import os
import tempfile
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from termcolor import colored

In [8]:
# Temp storage
tmp_fld = tempfile.mkdtemp(suffix='wdbc_data', prefix='tmp')
url_dta_fld = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/'
site = urlopen(url_dta_fld)
content = site.read()
soup = BeautifulSoup(content, "html.parser")
list_urls = soup.find_all('a')
# Download desired files
dta_fls = []
for url in list_urls:
    asst_url = urljoin(url_dta_fld,url['href'])
    if asst_url.endswith(('.data', '.names')):
        dta_fls.append(os.path.join(tmp_fld, asst_url.split('/')[-1]))
        print(asst_url)
        urlretrieve(url=asst_url, filename=dta_fls[-1])
        print(colored('Saved: ' + dta_fls[-1], 'green'))

https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data
Saved: /var/folders/7x/kwc1y_l96t55_rwlv35mg8xh0000gn/T/tmperolz9s4wdbc_data/breast-cancer-wisconsin.data
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names
Saved: /var/folders/7x/kwc1y_l96t55_rwlv35mg8xh0000gn/T/tmperolz9s4wdbc_data/breast-cancer-wisconsin.names
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
Saved: /var/folders/7x/kwc1y_l96t55_rwlv35mg8xh0000gn/T/tmperolz9s4wdbc_data/wdbc.data
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names
Saved: /var/folders/7x/kwc1y_l96t55_rwlv35mg8xh0000gn/T/tmperolz9s4wdbc_data/wdbc.names
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data
Saved: /var/folders/7x/kwc1y_l96t55_rwlv35mg8xh0000gn/T/tmperolz9s4wdbc_data/wpbc.data
https://archive.ics.uci.e